##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras 예제에서 잘라내기

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">깃허브(GitHub) 소스 보기</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/model_optimization/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 개요

규모 기반 *가중치 잘라내기*에 대한 엔드 투 엔드 예제를 시작합니다.

### 다른 페이지

잘라내기가 무엇인지에 대한 소개와 이를 사용해야 하는지를 결정하려면(지원되는 항목 포함) [개요 페이지](https://www.tensorflow.org/model_optimization/guide/pruning)를 참조하세요.

사용 사례에 필요한 API를 빠르게 찾으려면(희소성이 80%인 모델을 완전히 잘라내는 것 이상), [종합 가이드](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md)를 참조하세요.

### 요약

이 튜토리얼에서는 다음을 수행합니다.

1. MNIST용 `tf.keras` 모델을 처음부터 훈련합니다.
2. 잘라내기 API를 적용하여 모델을 미세 조정하고 정확성을 확인합니다.
3. 잘라내기로 3배 더 작은 TF 및 TFLite 모델을 만듭니다.
4. 잘라내기와 훈련 후 양자화를 결합하여 10배 더 작은 TFLite 모델을 만듭니다.
5. TF에서 TFLite까지 정확성의 지속성을 확인합니다.

## 설정

In [ ]:
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

## 잘라내기 없이 MNIST에 대한 모델 훈련하기

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

기준 테스트 정확성을 평가하고 나중에 사용할 수 있도록 모델을 저장합니다.

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

## 잘라내기로 사전 훈련된 모델 미세 조정하기


### 모델 정의하기

전체 모델에 잘라내기를 적용하고 모델 요약에서 이를 확인합니다.

이 예에서는 50% 희소성(가중치가 50% 0)으로 모델을 시작하고 80% 희소성으로 끝납니다.

[종합 가이드](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md)에서 모델 정확성의 향상을 위해 일부 레이어를 잘라내는 방법을 볼 수 있습니다.

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

### 기준선과 비교하여 모델 훈련 및 평가하기

두 epoch 동안 잘라내기로 미세 조정합니다.

`tfmot.sparsity.keras.UpdatePruningStep`은 훈련 중에 필요하며, `tfmot.sparsity.keras.PruningSummaries`는 진행률 추적 및 디버깅을 위한 로그를 제공합니다.

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

이 예제의 경우, 기준선과 비교하여 잘라낸 후 테스트 정확성의 손실이 최소화됩니다.

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

로그는 레이어별로 희소성의 진행률을 보여줍니다.

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

Colab이 아닌 사용자의 경우, [TensorBoard.dev](https://tensorboard.dev/experiment/sRQnrycaTMWQOaswXzClYA/#scalars&_smoothingWeight=0)에서 이 코드 블록의 [이전 실행의 결과](https://tensorboard.dev/)를 볼 수 있습니다.

## 잘라내기로 3배 더 작은 모델 생성하기

잘라내기의 압축 이점을 확인하려면, `tfmot.sparsity.keras.strip_pruning`과 표준 압축 알고리즘(예: gzip을 통해)을 적용하는 것이 모두 필요합니다.

- `strip_pruning`은 훈련 중에 잘라내기에만 필요한 모든 tf.Variable을 제거하기 때문에 필요합니다. 그렇지 않으면, 추론 중에 모델 크기가 추가됩니다.
- 직렬화된 가중치 행렬은 잘라내기 전과 크기가 같으므로 표준 압축 알고리즘을 적용해야 합니다. 그러나 잘라내기는 대부분의 가중치를 0으로 만들고, 알고리즘이 모델을 추가로 압축하는 데 사용할 수 있는 중복성이 추가됩니다.

먼저, TensorFlow를 위한 압축 가능한 모델을 만듭니다.

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

그런 다음 TFLite용 압축 가능한 모델을 만듭니다.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

실제로 gzip을 통해 모델을 압축하는 도우미 함수를 정의하고 압축된 크기를 측정합니다.

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

잘라내기로 3배 더 작아진 모델을 비교하고 확인합니다.

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

## 잘라내기와 양자화를 결합하여 10배 더 작은 모델 생성하기

추가 이점을 위해 잘라낸 모델에 훈련 후 양자화를 적용할 수 있습니다.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

## TF에서 TFLite까지 정확성의 지속성 확인하기

테스트 데이터세트에서 TF Lite 모델을 평가하는 도우미 함수를 정의합니다.

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

잘라낸 양자화 모델을 평가하고 TensorFlow의 정확성이 TFLite 백엔드까지 유지되는지 확인합니다.

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

## 결론

이 튜토리얼에서는 TensorFlow 및 TFLite용 TensorFlow Model Optimization Toolkit API를 사용하여 희소 모델을 만드는 방법을 살펴보았습니다. 그런 다음 추가 이점을 위해 잘라내기와 훈련 후 양자화를 결합했습니다.

정확성 차이를 최소화하면서 MNIST용으로 10배 더 작은 모델을 만들었습니다.

이 새로운 기능은 리소스가 제한된 환경에서 배포할 때 특히 중요하므로 사용해 볼 것을 권장합니다.
